In [468]:
LLM_QUERY = "I want easy comforting and savory food for lunch"
# LLM_MODEL = "llama3.2-vision:90b"
# LLM_MODEL = "gemma2:27b"
LLM_MODEL = "llama3.1:70b"
# LLM_MODEL = "llama3.2:latest"
# LLM_MODEL = "llama3.2-vision:90b"

In [469]:
query='''
"Egg", 
"Tomato", 
"Beans", 
"Tuna", 
"Farro", 
"Olive"
'''



# Print the data
# print(data)

def query_formatter(
                     query:str
                     ) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Ensure that context_items is a list of multiple summaries
    # context_summaries = [item["Course Summary"] for item in context_items]

    # Join context summaries into one paragraph with each item on a new line
    # context_text = "- " + "\n- ".join(context_summaries)
    
    # Build the prompt
    new_prompt = f'''
    Based on the user's ingredients: 
    \n {query},
    \n Imagine you are a professional chef with 30 years of global culinary experience. Your task is to generate a concise keyword from this ingredient list to help search for recipe suggestions online. The keyword does not need to specify a recipe type but should focus on the ingredients, prioritizing the most versatile or defining ones.
    \n Select a primary ingredient that could anchor the dish and create a well-balanced flavor profile.
    Optimize the search term to use only 5 words, without punctuation, and focus on essential ingredients.
    Provide the keyword within quotation marks in the following format: "keyword one keyword two keyword three keyword four keyword five"
    Just gve the answer within the quotation marks, no need addional text or information.
    '''
    # new_prompt = f'''
    # \n Based on the user's ingredients: 
    # \n {query},
    # \n Act as a celebrity chef with 30 years of experience. Create the best recipe search keyword from a given user query. Analyze all provided recipes and ingredients to select a primary ingredient and optimize the search with only 5 words, without punctuation, reply with only 5 words. Make the your created search keyword inside a quotation.
    # '''
    return new_prompt

# Format prompt with context items
new_prompt = query_formatter(query=query,
                                )

# Print the new prompt for debugging
print(new_prompt)


    Based on the user's ingredients: 
    
 
"Egg", 
"Tomato", 
"Beans", 
"Tuna", 
"Farro", 
"Olive"
,
    
 Imagine you are a professional chef with 30 years of global culinary experience. Your task is to generate a concise keyword from this ingredient list to help search for recipe suggestions online. The keyword does not need to specify a recipe type but should focus on the ingredients, prioritizing the most versatile or defining ones.
    
 Select a primary ingredient that could anchor the dish and create a well-balanced flavor profile.
    Optimize the search term to use only 5 words, without punctuation, and focus on essential ingredients.
    Provide the keyword within quotation marks in the following format: "keyword one keyword two keyword three keyword four keyword five"
    Just gve the answer within the quotation marks, no need addional text or information.
    


In [470]:
import requests
import json





headers = {
    "Content-Type": "application/json"
}

data = {
    "model": LLM_MODEL,
    "prompt": new_prompt,
    "stream": False,
    "options": {"num_ctx": 16000}
}

# data = {
#     "model": "llama3.1:70b",
#     "prompt": new_prompt,
#     "stream": False,
#     "options": {"num_ctx": 16000}
# }
print("starting")
response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.status_code)


# print(json.loads(response.text)["response"])

add = str(json.loads(response.text)["response"]).lower()
print(add)

starting
200
"tuna farro tomato olive egg"


In [471]:
import re

def extract_quoted_text(add):
    # Regular expression to find the first occurrence of text within double quotes
    match = re.search(r'"(.*?)"', add)
    if match:
        return match.group(1)  # Return only the first match
    else:
        return '' 


add = extract_quoted_text(add)
print(add)


tuna farro tomato olive egg


In [489]:
add = "tuna" ## CHATGPT ONLY

In [490]:
import requests
from bs4 import BeautifulSoup
import json

class RecipeScraper:
    def __init__(self, query):
        self.query = query
        self.recipes = []  # To store basic recipe info (title and link)
        self.all_recipes_details = []  # To store detailed recipe info
    
    def search_recipes(self):
        """Search for recipes on Cookpad using the provided query."""
        url = f'https://cookpad.com/us/search/{self.query.replace(" ", "%20")}'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
        }
        
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print("Failed to retrieve data. Status code:", response.status_code)
            return
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for recipe in soup.select('a.block-link__main'):
            if recipe:
                title = recipe.text.strip()
                link = 'https://cookpad.com' + recipe['href']
                self.recipes.append({'title': title, 'link': link})
        
        print(f"Found {len(self.recipes)} recipes for query '{self.query}'.")

    def get_recipe_details(self, url):
        """Fetch detailed recipe information from a given recipe link."""
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve recipe details. Status code: {response.status_code}")
            return {}

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract introduction
        intro_elem = soup.select_one('.text-cookpad-14')  # Adjust based on actual page structure
        introduction = intro_elem.text.strip() if intro_elem else "No introduction available"
        
        # Extract ingredients
        ingredients = [ingredient.text.strip() for ingredient in soup.select('.ingredient-list')]
        
        # Extract instructions (steps to cook)
        instructions = [step.text.strip() for step in soup.select('.step')]
        
        return {
            'introduction': introduction,
            'ingredients': ingredients,
            'instructions': instructions
        }

    def fetch_all_recipe_details(self):
        """Loop through all recipes found and fetch detailed information."""
        for idx, recipe in enumerate(self.recipes):
            print(f"Fetching details for {idx+1}: {recipe['title']}")
            details = self.get_recipe_details(recipe['link'])
            
            if details:
                recipe_data = {
                    "title": recipe['title'],
                    "introduction": details['introduction'],
                    "ingredients": details['ingredients'],
                    "instructions": details['instructions']
                }
                self.all_recipes_details.append(recipe_data)
    
    def save_all_recipes_to_json(self, filename="all_recipes.json"):
        """Save all recipes with details to a single JSON file."""
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(self.all_recipes_details, file, indent=4, ensure_ascii=False)
        
        print(f"All recipes saved to {filename}")

# Example usage
if __name__ == "__main__":
    # query = "Shrimp"
    print(add)
    scraper = RecipeScraper(add)
    
    scraper.search_recipes()
    scraper.fetch_all_recipe_details()
    scraper.save_all_recipes_to_json()

tuna
Found 30 recipes for query 'tuna'.
Fetching details for 1: Tuna
Fetching details for 2: Tuna Sisig
Fetching details for 3: California Farm Spicy Tuna Steak Apricot Salad
Fetching details for 4: California Farm Scalloped Carrots with Grilled Ahi Tuna Steaks
Fetching details for 5: California Farm Albacore Tuna Pate
Fetching details for 6: Tuna Salad
Fetching details for 7: California Farm Raw Tuna Toppings
Fetching details for 8: Quick Summer Tuna Pasta Salad
Fetching details for 9: Tuna Macaroni Salad
Fetching details for 10: Crispy Cheesy Tuna Pie
Fetching details for 11: California Farm Tuna Apple Cucumber Salad
Fetching details for 12: Tuna Melt
Fetching details for 13: Tuna Pockets
Fetching details for 14: California Farm Tuna Lettuce Spring Roll
Fetching details for 15: Easy Tuna Salad
Fetching details for 16: Tuna Broccoli Rotini
Fetching details for 17: Tuna salad
Fetching details for 18: Keto Cauliflower-Tuna Pizza
Fetching details for 19: Tuna pasta casserole
Fetching det

In [491]:
with open('all_recipes.json', 'r', encoding='utf-8') as file:
    recipes = json.load(file)

def clean_symbols(text_list):
    # Define unwanted symbols/patterns
    unwanted_patterns = [
        r'◆+',  # Repeated ◆ characters
        r'[\U0001F600-\U0001F64F]',  # Emoticons
        r'[\U0001F300-\U0001F5FF]',  # Miscellaneous symbols and pictographs
        r'[\U0001F680-\U0001F6FF]',  # Transport and map symbols
        r'[\U0001F1E0-\U0001F1FF]',  # Flags
        r'[\U0001F900-\U0001F9FF]',  # Supplemental Symbols and Pictographs
        r'[\U0001FA70-\U0001FAFF]',  # Symbols and Pictographs Extended-A
        r'[☀-⛿]',  # Miscellaneous Symbols
        r'[✀-➿]',  # Dingbats
        r'[一-鿿]',  # Chinese characters
        r'[\U0001F004-\U0001F0CF]'  # Mahjong and other symbols
    ]

    cleaned_list = []
    for text in text_list:
        cleaned_text = text
        for pattern in unwanted_patterns:
            cleaned_text = re.sub(pattern, '', cleaned_text)  # Remove unwanted symbols
        cleaned_list.append(cleaned_text.strip())  # Strip leading/trailing spaces
    
    return cleaned_list

def format_recipes(recipes):
    formatted_strings = []
    for recipe in recipes:
        ingredients = ', '.join(recipe['ingredients']).replace('\n', ' ')
        instructions = ' '.join(recipe['instructions']).replace('\n', ' ')
        formatted_recipe = f"The title is {recipe['title']}. The ingredients are: {ingredients}. The steps are: {instructions}."
        formatted_strings.append(formatted_recipe)
    
    # Clean the formatted strings from unwanted symbols
    return clean_symbols(formatted_strings)

# Example usage with recipes list
# Get the list of formatted recipes
formatted_recipes_list = format_recipes(recipes)

# Print each recipe as a long string
combined_recipes_list = []
for recipe_string in formatted_recipes_list:
    combined_recipes_list.append(recipe_string)

# Print each cleaned, formatted recipe string
# for combined_recipe in combined_recipes_list:
    # print(combined_recipe)

combined_recipes_list

['The title is Tuna. The ingredients are: 2 cans Oil Tuna   1 cup Chosen Foods Classic Mayo %100 Pure Avocado Oil   1/4 cup Vlasic Dill Relish   1/3 cup Stater Bros Tartar Sauce   1/4 The Original Japanese Barbecue Sauce by Bachan’s   1 1/4 cup Zero Sugar Mixed Berry Juice by Ocean Spray. The steps are: 1         Add tuna into a bowl 2         Add Chosen Foods Classic Mayo %100 Pure Avocado Oil 3         Add Vlasic Dill Relish 4         Add Stater Bros Tartar Sauce 5         Add The Original Japanese Barbecue Sauce by Bachan’s 6         Mix it. Stir. And stir well. 7         Have with a Zero Sugar Mixed Berry Juice by Ocean Spray.',
 "The title is Tuna Sisig. The ingredients are: 3 cans tuna   1 tbsp soy sauce   1/2 tbsp liquid seasoning    ground pepper- to taste   3 cloves garlic, minced   1 medium onion   2 pcs red chili   3 tbsp mayonnaise   1 tbsp margarine   1/4 cup chicharon for toppings   1 green chili for toppings   2 calamansi (optional). The steps are: 1         Place the oi

In [492]:

user_preference=f'''
    {LLM_QUERY}
'''

# Print the data
# print(data)

def prompt_formatter(user_preference:str,
                     query: str,
                     ) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Ensure that context_items is a list of multiple summaries
    # context_summaries = [item["Course Summary"] for item in context_items]

    # Join context summaries into one paragraph with each item on a new line
    # context_text = "- " + "\n- ".join(context_summaries)
    
    # Build the prompt
    new_prompt = f'''
    \n Based on the user's preference: 
    \n {user_preference},
    \n and considering these ingredients: 
    \n {query}
    \n As a 5-star celebrity chef with 30 years of experience, enhance the query to fully capture the user preference and ensure it highlights the most important ingredient. Only output the improved query directly, without any introductory text or additional context.    '''
    return new_prompt

# Format prompt with context items
new_prompt = prompt_formatter(user_preference=user_preference,
                              query=query,)

# Print the new prompt for debugging
print(new_prompt)


    
 Based on the user's preference: 
    
 
    I want easy comforting and savory food for lunch
,
    
 and considering these ingredients: 
    
 
"Egg", 
"Tomato", 
"Beans", 
"Tuna", 
"Farro", 
"Olive"

    
 As a 5-star celebrity chef with 30 years of experience, enhance the query to fully capture the user preference and ensure it highlights the most important ingredient. Only output the improved query directly, without any introductory text or additional context.    


In [493]:
import requests
import json



url = "http://localhost:11434/api/generate"

headers = {
    "Content-Type": "application/json"
}

data = {
    "model": LLM_MODEL,
    "prompt": new_prompt,
    "stream": False,
    "options": {"num_ctx": 16000}
}
# data = {
#     "model": "llama3.1:70b",
#     "prompt": new_prompt,
#     "stream": False,
#     "options": {"num_ctx": 16000}
# }
print("starting")
response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.status_code)


print(json.loads(response.text)["response"])

add = str(json.loads(response.text)["response"])

starting
200
I'm craving a simple yet satisfying lunch that's both comforting and savory. Specifically, I'd love something with tuna as the main ingredient, possibly paired with beans, tomato, and farro, all infused with the richness of olive flavor and topped with an egg, if possible.


In [494]:
# add = "easy banana milk breakfast recipe"

In [ ]:
with open('all_recipes.json', 'r', encoding='utf-8') as file:
    recipes = json.load(file)

# combined_recipes_list = []

def new_prompt_formatter(
                     combined_recipes_list: list[dict],
                     add:str
                     ) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Ensure that context_items is a list of multiple summaries
    # context_summaries = [item["Course Summary"] for item in context_items]

    # Join context summaries into one paragraph with each item on a new line
    # context_text = "- " + "\n- ".join(context_summaries)
    
    # Build the prompt
    new_prompt = f'''
As a seasoned chef with extensive culinary expertise, create a recipe tailored to the user's available ingredients and preferences, drawing inspiration from the provided recipes. Follow these guidelines:

1. **User Query**: Address the user's specific query or request:
   "{add}"

2. **Ingredients Focus**: Ensure the recipe centers around the user's available ingredients:
   {query}
   If essential ingredients are missing, suggest thoughtful substitutions that retain the dish’s original flavor, texture, and purpose.
   
# 3. **Inspirational Basis**: Base the recipe primarily on the provided recipe inspirations. Adapt techniques, flavor pairings, and structure from these inspirations:
#    {combined_recipes_list}

4. **Clear and Concise Instructions**:
   - Present the recipe in plain text.
   - Include a clear list of ingredients and step-by-step preparation instructions.
   - Avoid symbols, patterns, or bullet points; write in complete sentences.

5. **Substitution Suggestions**:
   - Provide substitutions for unavailable or challenging ingredients.
   - Ensure that suggested alternatives maintain the dish’s culinary intention.

6. **Consistency with Inspirations**:
   - Stay true to the style and techniques outlined in the inspirations.
   - Adapt only where necessary to align with the user's specific ingredients and preferences.

Deliver a fully structured recipe that is easy to understand and directly applicable. Ensure it incorporates substitutions where needed and offers any relevant tips or enhancements to elevate the dish.

<important>Create a recipe based on the user's query and ingredients, closely aligning with the techniques and ideas from the provided recipes.</important>
<todo>Provide a structured recipe, including ingredients, instructions, and thoughtful substitution suggestions in plain text format.</todo>
'''
#     new_prompt = f'''
# As a chef with 30 years of culinary expertise across diverse cuisines, create a recipe that perfectly aligns with the user's preference, focusing on the available ingredients and inspired by the provided recipe list:

# {add}

# These are the ingredients the user has:

# {query}

# Use the provided recipes primarily as references for techniques, flavor pairings, and overall dish structure:

# {combined_recipes_list}

# If essential ingredients are missing, suggest thoughtful substitutions that retain the dish’s original flavor, texture, and intention. The response must provide a clear, well-structured recipe including ingredients, step-by-step instructions, and substitution suggestions, presented in full sentences.

# Avoid using symbols, patterns, or bullet points. Write in plain text format. Ensure the recipe reflects the user's available ingredients and closely follows the inspirations provided, adapting only when necessary to suit the user's pantry.

# <important>Create a recipe based on the available ingredients, using the given recipes as inspiration.</important>
# <todo>Provide a recipe with ingredients, instructions, and thoughtful substitution suggestions.</todo>
# '''
    return new_prompt

# Format prompt with context items
new_prompt = new_prompt_formatter(add=add,
                              combined_recipes_list=combined_recipes_list,)

# Print the new prompt for debugging
print(new_prompt)


As a seasoned chef with extensive culinary expertise, create a recipe tailored to the user's available ingredients and preferences, drawing inspiration from the provided recipes. Follow these guidelines:

1. **User Query**: Address the user's specific query or request:
   "I want easy comforting and savory food for lunch"

2. **Ingredients Focus**: Ensure the recipe centers around the user's available ingredients:
   
"Egg", 
"Tomato", 
"Beans", 
"Tuna", 
"Farro", 
"Olive"

   If essential ingredients are missing, suggest thoughtful substitutions that retain the dish’s original flavor, texture, and purpose.

4. **Clear and Concise Instructions**:
   - Present the recipe in plain text.
   - Include a clear list of ingredients and step-by-step preparation instructions.
   - Avoid symbols, patterns, or bullet points; write in complete sentences.

5. **Substitution Suggestions**:
   - Provide substitutions for unavailable or challenging ingredients.
   - Ensure that suggested alternatives

In [497]:
combined_recipes_list

['The title is Tuna. The ingredients are: 2 cans Oil Tuna   1 cup Chosen Foods Classic Mayo %100 Pure Avocado Oil   1/4 cup Vlasic Dill Relish   1/3 cup Stater Bros Tartar Sauce   1/4 The Original Japanese Barbecue Sauce by Bachan’s   1 1/4 cup Zero Sugar Mixed Berry Juice by Ocean Spray. The steps are: 1         Add tuna into a bowl 2         Add Chosen Foods Classic Mayo %100 Pure Avocado Oil 3         Add Vlasic Dill Relish 4         Add Stater Bros Tartar Sauce 5         Add The Original Japanese Barbecue Sauce by Bachan’s 6         Mix it. Stir. And stir well. 7         Have with a Zero Sugar Mixed Berry Juice by Ocean Spray.',
 "The title is Tuna Sisig. The ingredients are: 3 cans tuna   1 tbsp soy sauce   1/2 tbsp liquid seasoning    ground pepper- to taste   3 cloves garlic, minced   1 medium onion   2 pcs red chili   3 tbsp mayonnaise   1 tbsp margarine   1/4 cup chicharon for toppings   1 green chili for toppings   2 calamansi (optional). The steps are: 1         Place the oi

In [496]:
import requests
import json



url = "http://localhost:11434/api/generate"

headers = {
    "Content-Type": "application/json"
}

data = {
    "model": LLM_MODEL,
    "prompt": new_prompt,
    "stream": False,
    "options": {"num_ctx": 25000}
}
# data = {
#     "model": "llama3.1:70b",
#     "prompt": new_prompt,
#     "stream": False,
#     "options": {"num_ctx": 16000}
# }
print("starting")
response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.status_code)


print(json.loads(response.text)["response"])

add = str(json.loads(response.text)["response"])

starting


KeyboardInterrupt: 

In [ ]:
# add = '''


# Japanese-Style Low Sodium Garlic Pork Fried Rice

# Ingredients: 1 cup day-old rice (preferably jasmine or Japanese rice) 100 grams pork loin, thinly sliced 2 tablespoons vegetable oil, divided 3 cloves garlic, minced 1 small onion, diced 1/2 cup green beans, chopped 1/2 cup carrots, diced 2 large eggs 1 tablespoon low-sodium soy sauce 1/2 teaspoon sesame oil 2 spring onions, chopped (for garnish) Salt and pepper to taste

# Instructions:

# 1. Prepare the pork by marinating it briefly in half a tablespoon of low-sodium soy sauce for extra flavor. Set aside.
# 2. Heat one tablespoon of oil in a large frying pan or wok over medium-high heat. Add the garlic and onion, stirring until fragrant and softened, about two minutes.
# 3. Add the sliced pork and cook until browned and fully cooked, about five minutes. Remove the pork and set it aside.
# 4. In the same pan, add another half tablespoon of oil and scramble the eggs until just set. Transfer the eggs to a bowl and set aside.
# 5. Add the remaining oil to the pan and toss in the green beans and carrots. Stir-fry the vegetables until tender-crisp, about three to four minutes.
# 6. Add the rice to the pan, breaking up any clumps. Drizzle with low-sodium soy sauce and sesame oil, stirring to combine and heat through for three minutes.
# 7. Return the pork and eggs to the pan, stirring everything to distribute evenly. Adjust seasoning with salt and pepper to taste.
# 8. Garnish with chopped spring onions and serve hot.
# Suggested Substitutions:

# For a vegetarian version, replace pork with tofu or mushrooms.
# If low-sodium soy sauce is unavailable, dilute regular soy sauce with water in a 1:1 ratio.
# If jasmine or Japanese rice is unavailable, substitute with any day-old white rice.

# '''

In [ ]:
# def new_prompt_formatter(
#                      add:str
#                      ) -> str:
#     """
#     Augments query with text-based context from context_items.
#     """
#     # Ensure that context_items is a list of multiple summaries
#     # context_summaries = [item["Course Summary"] for item in context_items]

#     # Join context summaries into one paragraph with each item on a new line
#     # context_text = "- " + "\n- ".join(context_summaries)
    
#     # Build the prompt
#     new_prompt = f'''
#     \n Generate an image that resembles the result from this recipe below and don't deviate from the ingredients that it has.
#     \n The recipes are :
#     \n {add}
    
#     '''
#     return new_prompt

# # Format prompt with context items
# new_prompt = new_prompt_formatter(add=add)

# # Print the new prompt for debugging
# print(new_prompt)

In [ ]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [ ]:
#!pip install ragas
# !pip install llama-index

In [ ]:
# from datasets import Dataset

# # Define the data
# questions = "As a professional chef with 30 years of experience across the world and Ph.D in Nutritionist, can you recommend me a recipe that is suitable with the user preference and with these recipes as your reference?"
# answers = [add]  # Ensure 'add' is a list of strings or the correct data format
# context = ["Hi! I want to eat Japanese-style cuisine but currently I have high blood pressure.", "Egg, Pork, Cabbage, Corn, Green Beans, Carrot, Beans"]
# recipes = [scraper.recipes]  # Example recipes
# references = ["A recipe recommendation that aligns with the user's preferences and health considerations."]  # Add references

# # Create the data dictionary
# data = {
#     "question": [questions],
#     "answer": answers,  # Ensure this matches the expected format
#     "contexts": [recipes],
#     "ground_truths": ["Egg, Pork, Cabbage, Corn, Green Beans, Carrot, Beans"],  # Should be a list of lists or strings
#     "reference": references  # Add this to match the required column
# }

# # Convert dict to dataset
# dataset = Dataset.from_dict(data)

# # Print the dataset to verify
# print(dataset)

In [ ]:
def tidy_up_string(add):
    # Remove excessive line breaks and ensure the string is formatted cleanly
    cleaned_string = add.replace('\n', ' ').replace('  ', ' ').strip()
    return cleaned_string

# Tidy up the input text
our_answer = add
tidied_text = tidy_up_string(add)

# Print the cleaned result
print(tidied_text)

**Comforting Tuna and Farro Bowl with Egg, Tomato, and Beans** This hearty bowl combines the user's available ingredients to create a satisfying and savory meal that perfectly suits their request for easy comforting food. **Ingredients:** * 1 cup Farro * 1 can Tuna (drained and flaked) * 2 Eggs * 2 medium Tomatoes, diced * 1/2 cup cooked Beans (such as cannellini or kidney beans) * 2 tablespoons Olive oil * Salt and pepper to taste **Instructions:** To prepare this comforting bowl, follow these steps: 1. **Cook the Farro:** Rinse the farro in a fine-mesh strainer under cold water, then drain well. In a medium saucepan, bring 4 cups of water to a boil. Add the farro and reduce heat to a simmer. Cook for about 20-25 minutes or until the farro is tender but still slightly chewy. Drain excess water. 2. **Prepare the Tuna:** In a bowl, mix together the flaked tuna, salt, and pepper. 3. **Cook the Eggs:** In a non-stick skillet, heat one tablespoon of olive oil over medium heat. Crack in the

In [ ]:
def clean_symbols(text_list):
    # Define unwanted symbols/patterns
    unwanted_patterns = [
        r'◆+',  # Repeated ◆ characters
        r'[\U0001F600-\U0001F64F]',  # Emoticons
        r'[\U0001F300-\U0001F5FF]',  # Miscellaneous symbols and pictographs
        r'[\U0001F680-\U0001F6FF]',  # Transport and map symbols
        r'[\U0001F1E0-\U0001F1FF]',  # Flags
        r'[\U0001F900-\U0001F9FF]',  # Supplemental Symbols and Pictographs
        r'[\U0001FA70-\U0001FAFF]',  # Symbols and Pictographs Extended-A
        r'[☀-⛿]',  # Miscellaneous Symbols
        r'[✀-➿]',  # Dingbats
        r'[一-鿿]',  # Chinese characters
        r'[\U0001F004-\U0001F0CF]'  # Mahjong and other symbols
    ]

    cleaned_list = []
    for text in text_list:
        cleaned_text = text
        for pattern in unwanted_patterns:
            cleaned_text = re.sub(pattern, '', cleaned_text)  # Remove unwanted symbols
        cleaned_list.append(cleaned_text.strip())  # Strip leading/trailing spaces
    
    return cleaned_list

def format_recipes(recipes):
    formatted_strings = []
    for recipe in recipes:
        ingredients = ', '.join(recipe['ingredients']).replace('\n', ' ')
        instructions = ' '.join(recipe['instructions']).replace('\n', ' ')
        formatted_recipe = f"The title is {recipe['title']}. The ingredients are: {ingredients}. The steps are: {instructions}."
        formatted_strings.append(formatted_recipe)
    
    # Clean the formatted strings from unwanted symbols
    return clean_symbols(formatted_strings)

# Example usage with recipes list
# Get the list of formatted recipes
formatted_recipes_list = format_recipes(recipes)

# Print each recipe as a long string
combined_recipes_list = []
for recipe_string in formatted_recipes_list:
    combined_recipes_list.append(recipe_string)

# Print each cleaned, formatted recipe string
# for combined_recipe in combined_recipes_list:
    # print(combined_recipe)

combined_recipes_list

[]

In [ ]:
formatted_recipes_list

[]

In [ ]:
# answer= '''
# \n Japanese-Style Garlic Pork Stir-Fry Recipe (Low-Sodium)
# \n 
# \n This recipe combines thinly sliced pork, crunchy vegetables, and a savory garlic sauce to create a delicious and healthy Japanese-inspired dish.
# \n 
# \n Ingredients:
# \n 
# \n 200g thinly sliced pork
# \n 3 cloves of garlic, minced
# \n 1 medium carrot, peeled and grated
# \n 100g green beans, trimmed
# \n 2 stalks of spring onions, chopped
# \n 1/4 cup low-sodium soy sauce
# \n 2 tablespoons rice vinegar
# \n 1 tablespoon sesame oil
# \n Salt-free seasoning blend (e.g., herbs and spices)
# \n Instructions:
# \n 
# \n 1. In a small bowl, whisk together soy sauce, rice vinegar, and sesame oil to make the garlic sauce.
# \n 2. Heat 1 tablespoon of sesame oil in a large skillet or wok over medium-high heat.
# \n 3. Add the minced garlic and cook for 1 minute until fragrant.
# \n 4. Add the sliced pork and cook until browned, about 3-4 minutes.
# \n 5. Add the grated carrot, green beans, and chopped spring onions to the skillet. Stir-fry for 2-3 minutes until the vegetables are tender-crisp.
# \n 6. Pour the garlic sauce into the skillet and stir-fry for an additional minute.
# \n 7. Season with salt-free seasoning blend to taste.
# \n 8. Serve hot over brown rice or noodles.
# \n 
# \n Alternative Ingredients:
# \n 
# \n For pork, you can substitute chicken breast or thighs, sliced into thin strips.
# \n For green beans, you can use snap peas or bok choy as alternatives.
# \n For spring onions, you can use shallots or leeks instead.
# \n 
# \n This recipe is carefully crafted to minimize sodium content while maintaining the bold flavors of Japanese cuisine. Enjoy your delicious and healthy garlic pork stir-fry!
# \n 
# '''

In [ ]:
gpt_answer = """
### Easy Comforting and Savory Lunch Recipe: Tuna and Tomato Farro Salad

**Introduction:**
This recipe combines the core elements from the user's available ingredients and inspiration from the suggested recipes. It features a hearty and savory dish that is simple to prepare, comforting, and perfect for lunch.

---

**Ingredients:**
- 1 cup cooked farro (or substitute with quinoa or rice if unavailable)
- 1 can tuna, drained
- 1 large tomato, diced
- 1/2 cup cooked beans (any type you have)
- 2 eggs, boiled and sliced
- 1/4 cup olives, sliced
- 2 tablespoons olive oil
- 1 tablespoon lemon juice (or vinegar as a substitute)
- 1 teaspoon dried oregano
- Salt and pepper to taste

---

**Instructions:**

1. **Prepare the Ingredients:**
   - Cook the farro according to package instructions. Drain and let it cool slightly.
   - Dice the tomato, slice the olives, and boil the eggs for 8-10 minutes. Peel and slice the eggs.

2. **Combine the Salad Base:**
   - In a large bowl, combine the cooked farro, diced tomato, beans, and tuna. Use a fork to gently break apart the tuna into smaller chunks.

3. **Make the Dressing:**
   - In a small bowl, whisk together olive oil, lemon juice, oregano, salt, and pepper.

4. **Assemble the Salad:**
   - Pour the dressing over the salad mixture and toss gently to coat all ingredients evenly.
   - Arrange the sliced eggs and olives on top as a garnish.

5. **Serve:**
   - Serve the salad at room temperature or slightly chilled. It’s a comforting and balanced meal that’s perfect for lunch.

---

**Tips:**
- For extra flavor, you can add a sprinkle of grated cheese or a handful of fresh herbs like parsley or basil.
- If you prefer a warm dish, heat the salad gently in a pan before adding the boiled eggs and olives.

Enjoy your easy and savory lunch!
"""

In [511]:
out_answer = """
### Easy Comforting and Savory Lunch Recipe: Tuna and Tomato Farro Salad

**Introduction:**
This recipe combines the core elements from the user's available ingredients and inspiration from the suggested recipes. It features a hearty and savory dish that is simple to prepare, comforting, and perfect for lunch.

---

**Ingredients:**
- 1 cup cooked farro (or substitute with quinoa or rice if unavailable)
- 1 can tuna, drained
- 1 large tomato, diced
- 1/2 cup cooked beans (any type you have)
- 2 eggs, boiled and sliced
- 1/4 cup olives, sliced
- 2 tablespoons olive oil
- 1 tablespoon lemon juice (or vinegar as a substitute)
- 1 teaspoon dried oregano
- Salt and pepper to taste

---

**Instructions:**

1. **Prepare the Ingredients:**
   - Cook the farro according to package instructions. Drain and let it cool slightly.
   - Dice the tomato, slice the olives, and boil the eggs for 8-10 minutes. Peel and slice the eggs.

2. **Combine the Salad Base:**
   - In a large bowl, combine the cooked farro, diced tomato, beans, and tuna. Use a fork to gently break apart the tuna into smaller chunks.

3. **Make the Dressing:**
   - In a small bowl, whisk together olive oil, lemon juice, oregano, salt, and pepper.

4. **Assemble the Salad:**
   - Pour the dressing over the salad mixture and toss gently to coat all ingredients evenly.
   - Arrange the sliced eggs and olives on top as a garnish.

5. **Serve:**
   - Serve the salad at room temperature or slightly chilled. It’s a comforting and balanced meal that’s perfect for lunch.

---

**Tips:**
- For extra flavor, you can add a sprinkle of grated cheese or a handful of fresh herbs like parsley or basil.
- If you prefer a warm dish, heat the salad gently in a pan before adding the boiled eggs and olives.

Enjoy your easy and savory lunch!
"""

In [ ]:
recipes = [
    {
        "title": "Savory! Tuna Potato Pie with Bechamel Sauce",
        "introduction": "To start creating your recipe library, please register or login.",
        "ingredients": [
            "Shortcrust Pastry",
            "60 g (2.1 oz, 1/2 us cup) bread flour",
            "60 g (2.1 oz, 1/2 us cup) cake flour",
            "1 g (1/5 tsp) salt",
            "60 g (2.1 oz) salted butter",
            "60 g (2.1 oz) cold water",
            "Bechamel Sauce",
            "20 g (0.7 oz, 1.7 Tbsp) salted butter",
            "20 g (0.7 oz, 2.2 Tbsp) cake flour, or all-purpose flour",
            "160 g (5.6 oz, 7/10 us cup) soy milk, or milk",
            "1/2 cube chicken broth",
            "1 Tbsp boiling water, for chicken broth",
            "pepper",
            "nutmeg powder",
            "Filling",
            "1 potato (about 120g, 4.2 oz, peeled)",
            "1/4 onion (about 40-50g, 1.4-1.7 oz)",
            "1 canned tuna, drained weight 50g / 1.75 oz",
            "salt and pepper",
            "olive oil for stir-frying",
            "Topping",
            "grated cheese"
        ],
        "instructions": [
            "Stir bread flour, cake flour, and salt. Dice butter and chill all ingredients for the crust.",
            "Rub butter into flour to form a breadcrumb texture. Add water and fold until large clumps form.",
            "Flatten dough, wrap in plastic, and refrigerate overnight.",
            "Prepare bechamel sauce by melting butter, adding flour, and mixing until smooth. Gradually add soy milk and broth.",
            "Slice potato, onion, and mix with drained tuna. Cook and season.",
            "Roll out pastry, blind bake, and add filling with bechamel sauce and cheese.",
            "Bake at 180°C (356°F) for 20 minutes."
        ]
    },
    {
        "title": "Savory Eggplant Stew",
        "introduction": "To start creating your recipe library, please register or login.",
        "ingredients": [
            "2 long eggplants",
            "1 can diced tomato",
            "4 garlic cloves",
            "1/4 green onion",
            "1/4 cup dried red pepper",
            "2 tbsp oil",
            "2 tbsp soy sauce",
            "1 tsp sugar",
            "1 tsp vinegar"
        ],
        "instructions": [
            "Preheat oven to 350°F.",
            "Cut eggplants into strips, bake for 10 minutes.",
            "Mince garlic and chop green onion.",
            "Heat oil, add garlic, green onion, and dried red pepper.",
            "Add diced tomato, soy sauce, sugar, and vinegar. Cook to boil.",
            "Add baked eggplant and cook for 8-10 minutes."
        ]
    },
    {
        "title": "Steamed Savory Egg Custard (水蒸蛋)",
        "introduction": "To start creating your recipe library, please register or login.",
        "ingredients": [
            "2 large pasture-raised eggs",
            "8 shell-less shrimps",
            "12 oz water",
            "1 tsp starch",
            "2 tsp fish sauce",
            "Sesame oil for drizzling"
        ],
        "instructions": [
            "Marinate shrimp in fish sauce for 3 minutes. Coat with starch and place in a large bowl.",
            "Beat eggs, gradually adding water until smooth.",
            "Steam in a water bath at 350°F for 30-35 minutes until set."
        ]
    }
]

# combined_recipes = []

# for recipe in recipes:
#     combined_recipe = ""
#     combined_recipe += f"Title: {recipe['title']}\n"
#     combined_recipe += "Introduction:\n"
#     combined_recipe += f"{recipe['introduction']}\n\n"
#     combined_recipe += "Ingredients:\n"
#     combined_recipe += "\n".join(recipe["ingredients"])
#     combined_recipe += "\n\nInstructions:\n"
#     combined_recipe += "\n".join(recipe["instructions"])
#     combined_recipe += "\n" + "="*40 + "\n\n"
    
#     combined_recipes.append(combined_recipe)

# # Print or use the combined variable as needed
# print(combined_recipes)
# len(combined_recipes)


# Convert recipes into plain text strings
# retrieved_contexts = []

# for recipe in recipes:
#     title = recipe.get("title", "No Title")
#     introduction = recipe.get("introduction", "No Introduction Available")
#     ingredients = recipe.get("ingredients", [])
#     instructions = recipe.get("instructions", [])
    
#     # Format the recipe into plain text
#     recipe_text = f"Title: {title}\n"
#     recipe_text += f"Introduction:\n{introduction}\n\n"
#     recipe_text += "Ingredients:\n"
#     recipe_text += "\n".join(ingredients) + "\n\n"
#     recipe_text += "Instructions:\n"
#     recipe_text += "\n".join(instructions)
    
#     # Append the plain text string to the list
#     retrieved_contexts.append(recipe_text)

formatted_recipes = []

for recipe in recipes:
    formatted_recipe = (
        f"The title is {recipe['title']}. "
        f"The ingredients are: {', '.join(recipe['ingredients'])}. "
        # f"The steps are: {' '.join(recipe['instructions'])}."
    )
    formatted_recipes.append(formatted_recipe)

# The combined formatted recipes in one variable
formatted_recipes_combined = formatted_recipes

# Ensure all contexts are valid strings
# print(retrieved_contexts)

In [524]:
from datasets import Dataset

# Define the data
questions = "As a seasoned chef with 30 years of culinary expertise around the globe, craft a recipe that perfectly aligns with the user's preference"
# answers = tidied_text  # Ensure 'add' is a list of strings or the correct data format
# answers = tidied_text  # Ensure 'add' is a list of strings or the correct data format
answers = out_answer  # Ensure 'add' is a list of strings or the correct data format
context = combined_recipes_list#"Hi! I want to eat Japanese-style cuisine but currently I have high blood pressure.", "The ingredients are Egg, Pork, Cabbage, Corn, Green Beans, Carrot, Beans"
# recipes = [recipes]  # Example recipes
# references = LLM_QUERY  # Add references
references = gpt_answer  # Add references

# Create the data dictionary
data = {
    "question": [questions],
    "answer": [answers],  # Ensure this matches the expected format
    "contexts": [context],
    # "ground_truths": ["Egg, Pork, Cabbage, Corn, Green Beans, Carrot, Beans"],  # Should be a list of lists or strings
    "reference": [references]  # Add this to match the required column
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

# Print the dataset to verify
print(dataset)

Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 1
})


In [525]:
# from datasets import Dataset

# # Define the data
# questions = "As a professional chef with 30 years of experience across the world and Ph.D in Nutritionist, can you recommend me a recipe that is suitable with the user preference and with these recipes as your reference?"
# answers = [add]  # Ensure 'add' is a list of strings or the correct data format
# context = ["Hi! I want to eat Japanese-style cuisine but currently I have high blood pressure."]
# recipes = [recipes]  # Example recipes
# references = ["A recipe recommendation that aligns with the user's preferences and health considerations."]  # Add references

# # Create the data dictionary
# data = {
#     "question": [questions],
#     "answer": answers,  # Ensure this matches the expected format
#     "contexts": [context],
#     "ground_truths": [recipes],  # Should be a list of lists or strings
#     "reference": references  # Add this to match the required column
# }

# # Convert dict to dataset
# dataset = Dataset.from_dict(data)

# # Print the dataset to verify
# print(dataset)

In [526]:
# !pip install llama-index-llms-ollama

In [527]:
import os
from llama_index.llms.ollama import Ollama
from langchain_ollama import OllamaEmbeddings

llm = Ollama(model="llama3.1:70b", request_timeout=10000.0)
embeddings = OllamaEmbeddings(model="llama3.1:70b")
# resp = llm.complete("Who is Paul Graham?")
# print(resp)
os.environ["OPENAI_API_KEY"]=""



In [528]:
import os
from ragas import RunConfig

os.environ["RAGAS_DEBUG"] = "true"

run_config = RunConfig(timeout=10000, log_tenacity=True) #, max_retries=50

In [529]:
print(type(tidied_text))  # Should match the expected type (e.g., str or list of str)
print(type(formatted_recipes_list))  # Should be a string or list, as needed
# print(type(data["ground_truths"]))  # Ensure this is a list of strings

<class 'str'>
<class 'list'>


In [530]:
from ragas import evaluate
from ragas.metrics import (
    #context_precision,
    #faithfulness,
    answer_relevancy,
    #context_recall,
    #context_entity_recall,
    answer_correctness,
    answer_similarity,
)

result = evaluate(
    dataset=dataset, 
    metrics=[
        #context_precision,
        #faithfulness,
        answer_relevancy,
        #context_recall,
        #context_entity_recall,
        answer_correctness,
        answer_similarity,
    ], llm=llm, embeddings=embeddings, run_config=run_config
)

df = result.to_pandas()
print(df)

Evaluating: 100%|██████████| 3/3 [02:16<00:00, 45.35s/it]


                                          user_input  \
0  As a seasoned chef with 30 years of culinary e...   

                                  retrieved_contexts  \
0  [The title is Tuna. The ingredients are: 2 can...   

                                            response  \
0  \n### Easy Comforting and Savory Lunch Recipe:...   

                                           reference  answer_relevancy  \
0  \n### Easy Comforting and Savory Lunch Recipe:...          0.089845   

   answer_correctness  semantic_similarity  
0                 NaN             0.834796  


In [531]:
# import os
# import torch
# from diffusers import StableDiffusion3Pipeline

# # Set CUDA_VISIBLE_DEVICES to 1 to use GPU 1
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-medium", torch_dtype=torch.bfloat16)
# pipe = pipe.to("cuda")

# image = pipe(
#     new_prompt,
#     num_inference_steps=28,
#     guidance_scale=3.5,
# ).images[0]
# image.save("food.png")


1. Generate 10 images
2. Feed to Vision LLM and ask it of for ranking based on the user's query
3. Retrieve The best from Vision LLM 

In [532]:
df

,user_input,retrieved_contexts,response,reference,answer_relevancy,answer_correctness,semantic_similarity
0,As a seasoned chef with 30 years of culinary e...,[The title is Tuna. The ingredients are: 2 can...,\n### Easy Comforting and Savory Lunch Recipe:...,\n### Easy Comforting and Savory Lunch Recipe:...,0.089845,NaN,0.834796


OTHER VERSION

In [533]:
from datasets import Dataset

# Define the data
questions = "As a seasoned chef with 30 years of culinary expertise around the globe, craft a recipe that perfectly aligns with the user's preference"
answers = [tidied_text]  # Ensure 'add' is a list of strings or the correct data format
context = [LLM_QUERY]
recipes = [formatted_recipes_list]  # Example recipes
references = ["A recipe recommendation that aligns with the user's preferences and health considerations."]  # Add references

# Create the data dictionary
data = {
    "question": [questions],
    "answer": answers,  # Ensure this matches the expected format
    "contexts": [context],
    # "ground_truths": ,  # Should be a list of lists or strings
    "reference": [combined_recipes_list]  # Add this to match the required column
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

# Print the dataset to verify
print(dataset)

Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 1
})


In [534]:
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    faithfulness,
    answer_relevancy,
    context_recall,
    context_entity_recall,
    answer_correctness,
    answer_similarity,
)



result = evaluate(
    dataset=dataset, 
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_entity_recall,
        answer_correctness,
        answer_similarity,
    ], llm=llm, embeddings=embeddings, run_config=run_config
)

df = result.to_pandas()
print(df)

ValidationError: 1 validation error for SingleTurnSample
reference
  Input should be a valid string [type=string_type, input_value=['The title is Tuna. The ...le then rolling over..'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

In [ ]:
df

In [ ]:
# # TODO:
# from ragas import evaluate, RunConfig
# from ragas.metrics import (
#     answer_relevancy,
#     faithfulness,
#     context_recall,
#     context_precision,
#     context_utilization,
#     context_entity_recall,
#     answer_correctness,
#     answer_similarity,
# )
# from ragas.metrics.critique import harmfulness
# from datasets import Dataset
# import pandas as pd

# # Load your dataset
# df = pd.read_excel('testset_full-200-ES.xlsx')
# data_samples = {
#     'question': df['question'].tolist(),
#     'answer': df['answer'].tolist(),
#     'contexts': df['contexts'].apply(lambda x: [x] if pd.notna(x) else []).tolist(),
#     'ground_truth': df['ground_truth'].tolist()
# }
# dataset = Dataset.from_dict(data_samples)

# # Create a RunConfig with a specified timeout
# run_config = RunConfig(timeout=120)  # Set timeout to 120 seconds

# # Use the evaluate function with the custom run_config
# result = evaluate(
#     dataset,
#     metrics=[
#         context_precision,
#         faithfulness,
#         answer_relevancy,
#         context_recall,
#         context_utilization,
#         context_entity_recall,
#         answer_correctness,
#         answer_similarity,
#         harmfulness,
#     ],
#     run_config=run_config
# )

# df = result.to_pandas()
# print(result)
# df.to_csv('results_es.csv', index=False)

In [ ]:
from datasets import Dataset

# Define the data
questions = "As a seasoned chef with 30 years of culinary expertise around the globe, craft a recipe that perfectly aligns with the user's preference"
answers = tidied_text  # Ensure 'add' is a list of strings or the correct data format
context = combined_recipes_list#"Hi! I want to eat Japanese-style cuisine but currently I have high blood pressure.", "The ingredients are Egg, Pork, Cabbage, Corn, Green Beans, Carrot, Beans"
# recipes = [recipes]  # Example recipes
references = LLM_QUERY  # Add references

# Create the data dictionary
data = {
    "question": [questions],
    "answer": [answers],  # Ensure this matches the expected format
    "contexts": [context],
    # "ground_truths": ["Egg, Pork, Cabbage, Corn, Green Beans, Carrot, Beans"],  # Should be a list of lists or strings
    "reference": [references]  # Add this to match the required column
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

# Print the dataset to verify
print(dataset)

In [ ]:
import os
from llama_index.llms.ollama import Ollama
from langchain_ollama import OllamaEmbeddings

llm = Ollama(model="llama3.1:70b", request_timeout=10000.0)
embeddings = OllamaEmbeddings(model="llama3.1:70b")
# resp = llm.complete("Who is Paul Graham?")
# print(resp)
os.environ["OPENAI_API_KEY"]=""


In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas.llms import LlamaIndexLLMWrapper
from ragas.embeddings import LlamaIndexEmbeddingsWrapper

# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# embeddings = HuggingfaceEmbeddings(model_name="bert-base-uncased")


from ragas import evaluate

evaluator_llm = LlamaIndexLLMWrapper(Ollama(model="llama3"))
evaluator_embeddings = LlamaIndexEmbeddingsWrapper(OllamaEmbeddings(model="llama3"))

metrics = [
    LLMContextRecall(llm=evaluator_llm), 
    FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]
# results = evaluate(dataset=eval_dataset, metrics=metrics)


result = evaluate(
    dataset=dataset, 
    metrics=metrics,
    llm=llm,
    embeddings=embeddings, 
    run_config=run_config
)

df = result.to_pandas()
print(df)